In [25]:
import tensorflow as tf
import numpy as np
import os

from sklearn.model_selection import train_test_split

import json

# Initial global var

In [26]:
## 미리 Global 변수를 지정하자. 파일 명, 파일 위치, 디렉토리 등이 있다.

Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
LABEL_TRAINING_DATA_FILE = 'label_train.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'
DATA_PATH = './data/'

## 학습에 필요한 파라메터들에 대해서 지정하는 부분이다.

BATCH_SIZE = 128
EPOCH = 50
HIDDEN = 64
BUFFER_SIZE = 2048

TEST_SPLIT = 0.1
RNG_SEED = 13371447
EMBEDDING_DIM = 128
MAX_SEQ_LEN = 25

# Load Dataset

In [27]:
## 데이터를 불러오는 부분이다. 효과적인 데이터 불러오기를 위해, 미리 넘파이 형태로 저장시킨 데이터를 로드한다.

q1_data = np.load(open(DATA_PATH + Q1_TRAINING_DATA_FILE, 'rb'))
q2_data = np.load(open(DATA_PATH + Q2_TRAINING_DATA_FILE, 'rb'))
labels = np.load(open(DATA_PATH + LABEL_TRAINING_DATA_FILE, 'rb'))

with open(DATA_PATH + NB_WORDS_DATA_FILE, 'r') as f:
    prepro_configs = json.load(f)

# ## 미리 저장한 임베딩 값을 불러오자
# word_embedding_matrix = np.load(open(DATA_PATH+WORD_EMBEDDING_MATRIX_FILE, 'rb'))
# with open(DATA_PATH+NB_WORDS_DATA_FILE, 'r') as f:
#     nb_words = json.load(f)['nb_words']



In [28]:
VOCAB_SIZE = prepro_configs['vocab_size']

# Split train and test dataset

In [80]:
q1_data_len = np.array([min(len(x), MAX_SEQ_LEN) for x in q1_data])
q2_data_len = np.array([min(len(x), MAX_SEQ_LEN) for x in q2_data])

In [83]:
## 데이터를 나누어 저장하자. sklearn의 train_test_split을 사용하면 유용하다. 하지만, 쿼라 데이터의 경우는
## 입력이 1개가 아니라 2개이다. 따라서, np.stack을 사용하여 두개를 하나로 쌓은다음 활용하여 분류한다.

X = np.stack((q1_data, q2_data), axis=1) 
y = labels

# X = word_embedding_matrix[X[:1000]]
# y = y[:1000]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RNG_SEED)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

#     # Build the embedding matrix
#     for word, index in vocabs.items():
#         if word in word2vec.vocab:
#             embeddings[index] = word2vec.word_vec(word)
#     del word2vec

In [30]:
def rearrange(q, sim_q, labels):
    features = {"q": q, "sim_q": sim_q}
    return features, labels

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((Q1_train, Q2_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(Q1_train))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(rearrange)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((Q1_test, Q2_test, y_test))
    dataset = dataset.shuffle(buffer_size=len(Q1_test))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(rearrange)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [108]:
def Malstm(features, labels, mode, params):
    
    size = 2
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    labels = tf.to_float(tf.reshape(labels, [-1,1]))

    q1 = tf.contrib.layers.embed_sequence(
                                            features['q'],
                                            VOCAB_SIZE,
                                            EMBEDDING_DIM,
                                            initializer=params['embed_init']
                                            )

    q2 = tf.contrib.layers.embed_sequence(
                                            features['sim_q'],
                                            VOCAB_SIZE,
                                            EMBEDDING_DIM,
                                            initializer=params['embed_init']
                                            )
                
    with tf.variable_scope('q_bi_lstm'):
        
        lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_units = HIDDEN, activation = tf.nn.tanh)
        lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_units = HIDDEN, activation = tf.nn.tanh)
        q_outputs, q_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = lstm_fw_cell,
                                                          cell_bw = lstm_bw_cell,
                                                          inputs = q1,
                                                          dtype = tf.float32)
        
        q_concat = tf.concat([q_outputs[0], q_outputs[1]], axis=2)
        q_final_state = tf.keras.layers.Dense(128)(q_concat[:,-1,:])
#         hidden = q_concat[:,-1,:]
# #         hidden = tf.layers.dense(inputs=q_concat[:,-1,:], units=1, activation=tf.nn.tanh)
        
#         print(q_concat)
#         print(q_final_state)
        
#         q_final_state = tf.layers.dense(inputs=q_concat, units=128, activation=tf.nn.tanh)
        
#         print(q_final_state)
    
    with tf.variable_scope('sim_bi_lstm'):
        lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(num_units = HIDDEN, activation = tf.nn.tanh)
        lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(num_units = HIDDEN, activation = tf.nn.tanh)
        sim_outputs, sim_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = lstm_fw_cell,
                                                          cell_bw = lstm_bw_cell,
                                                          inputs = q2,
                                                          dtype = tf.float32)

        sim_concat = tf.concat([sim_outputs[0], sim_outputs[1]], axis=2)
        sim_final_state = tf.keras.layers.Dense(128)(sim_concat[:,-1,:])
        

#         sim_final_state = tf.layers.dense(inputs=sim_concat, units=128, activation=tf.nn.tanh)
        
#         sim_final_state = tf.layers.dense(inputs=sim_outputs[:,-1,:], units=HIDDEN, activation=tf.nn.tanh)
                
    with tf.variable_scope('output_layer'):
        #Malstm
        logits = K.exp(-K.sum(K.abs(q_final_state -  sim_final_state), axis=1, keepdims=True))
        print(logits)
            
    if TRAIN:
        global_step = tf.train.get_global_step()
        loss =tf.losses.sigmoid_cross_entropy(labels, logits)
#         loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)

#         loss = tf.keras.backend.binary_crossentropy(
#             labels,
#             logits,
#             from_logits=False
#         )
        
#         loss = tf.losses.mean_squared_error(labels, output)
#         loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.one_hot(labels, depth=2),
#                                                logits = output
#                                               )
        
        train_op = tf.train.AdamOptimizer(1e-4).minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.mean_squared_error(labels, output)
        pred = tf.nn.sigmoid(output)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': cos_sim,
                'q_sem': q_dense_layer,
                'sim_q_sem': sim_q_dense_layer
            }
        )
    
params = {'embed_init': tf.random_uniform_initializer(-1.0, 1.0)}

In [110]:
model_dir = os.path.join(os.getcwd(), "data/checkpoint/")
os.makedirs(model_dir, exist_ok=True)

config_tf = tf.estimator.RunConfig()
config_tf._save_checkpoints_steps = 100
config_tf._save_checkpoints_secs = None
config_tf._keep_checkpoint_max =  2
config_tf._log_step_count_steps = 5

dssm_est = tf.estimator.Estimator(Malstm, model_dir=model_dir, config=config_tf, params=params)
dssm_est.train(train_input_fn, steps=EPOCH)

INFO:tensorflow:Using config: {'_model_dir': '/Users/sinseongjin/github/DeepNLP/7.NLPBOOK/6.TEXT_SIM/data/checkpoint/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 5, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x127ed9c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Tensor("output_layer/Exp:0", shape=(?, 1), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorfl

KeyboardInterrupt: 

In [66]:
time_start = datetime.now()

# validation_monitor = tf.contrib

print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

# train_spec = tf.estimator.TrainSpec(train_input_fn)
# eval_spec = tf.estimator.EvalSpec(test_input_fn)

dssm_est = tf.estimator.Estimator(Malstm, model_dir=model_dir, config=config_tf, params=params)
dssm_est.train(train_input_fn, steps=EPOCH)

# tf.estimator.train_and_evaluate(dssm_est, train_spec, eval_spec)

# For prediction or extract values
# prediction = est.predict(eval_input_fn)

# for i, p in enumerate(prediction):
#     print(i, p['sim_q_sem'])

time_end = datetime.now()

print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))

Experiment started at 23:21:50
.......................................
1.10.1
INFO:tensorflow:Using config: {'_model_dir': '/Users/sinseongjin/github/DeepNLP/7.NLPBOOK/6.TEXT_SIM/data/checkpoint/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1527a3748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Tensor("q_bi_lstm/concat:0", shape=(?, 25, 128), dtype=float32)


NameError: name 'q_final_state' is not defined

In [ ]:
FILE_DIR_PATH = './'
Q1_TEST_DATA_FILE = 'q1_test.npy'
Q2_TEST_DATA_FILE = 'q2_test.npy'

test_q1_data = np.load(open(FILE_DIR_PATH + Q1_TEST_DATA_FILE, 'rb'))
test_q2_data = np.load(open(FILE_DIR_PATH + Q2_TEST_DATA_FILE, 'rb'))

predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"q":test_q1_data, 
                                                         "sim_q":test_q2_data}, 
                                                      shuffle=False)

predictions = np.array([p['prob'] for p in est.predict(input_fn=
predict_input_fn)])

In [ ]:
#cross entropy

p = tf.placeholder(tf.float32, shape=[None, 5])
logit_q = tf.placeholder(tf.float32, shape=[None, 5])
q = tf.nn.sigmoid(logit_q)

feed_dict = {
  p: [[0, 0, 0, 1, 0],
      [1, 0, 0, 0, 0]],
  logit_q: [[0.2, 0.2, 0.2, 0.2, 0.2],
            [0.3, 0.3, 0.2, 0.1, 0.1]]
}

prob1 = -p * tf.log(q)
# prob2 = p * -tf.log(q) + (1 - p) * -tf.log(1 - q)
# prob3 = p * -tf.log(tf.sigmoid(logit_q)) + (1-p) * -tf.log(1-tf.sigmoid(logit_q))
# prob4 = tf.nn.sigmoid_cross_entropy_with_logits(labels=p, logits=logit_q)
print(prob1.eval(feed_dict))
# print(prob2.eval(feed_dict))
# print(prob3.eval(feed_dict))
# print(prob4.eval(feed_dict))

In [ ]:
https://github.com/bradleypallen/keras-quora-question-pairs/blob/master/quora-question-pairs-data-prep.ipynb
https://github.com/Smerity/keras_snli
https://github.com/bradleypallen/keras-quora-question-pairs/blob/master/quora-question-pairs-training-attention.ipynb
https://github.com/bradleypallen/keras-quora-question-pairs/blob/master/quora-question-pairs-training.ipynb
http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/
